In [1]:
# Converting from Earth-Centered J2000 to MCMF based on DE421

# Library imports
import numpy as np
import spiceypy as sp
from os.path import join

# Local imports
from paths import BASE_DIR
from file_io_util import quick_cd
from Structures import Quaternion, quat_mult


In [2]:
# Kernel Setup
KERNEL_DIR=join(BASE_DIR,"spicedata")
METAKRNL=join(KERNEL_DIR,"boromir.tm")
with quick_cd(KERNEL_DIR):
	sp.furnsh(METAKRNL)

In [3]:
# Read in some data to be converted
def read_data(filename,column_dict,header=False):
	"""
		Takes in a csv file and a dictionary of which values are in which columns
	"""
	column_list = [
		column_dict["time"],
		column_dict["x"],
		column_dict["y"],
		column_dict["z"],
		column_dict["qs"],
		column_dict["qx"],
		column_dict["qy"],
		column_dict["qz"]
	]
	text_data = []
	with open(filename) as f:
		text_data = f.read().strip("\n").split("\n")
	data = [["" for _ in column_list] for _ in text_data]
	for i,line in enumerate(text_data):
		if header and i == 0:
			continue
		line_data = line.split(",")
		for j,entry in enumerate(column_list):
			data[i][j] = line_data[entry]
	return data
	

In [27]:
# Convert the data into MCMF
def convertEarthJ2000toMCMF(data):
	"""
		Convert data from Earth-centered J2000 to MCMF Frame
		
		data should be a 2-d array, where the inner array has the form:
			[time_utc,x,y,z,qs,qx,qy,qz]
	"""

	out_data = []

	for i,state in enumerate(data):
		out_entry = ["" for i in range(19)]
		out_entry[0] = "img{}".format(i)

		# Add time over
		date,time = state[0].split("T")
		utc_time = "{} {}".format(date.replace(":","-"),time)
		out_entry[8] = utc_time
		#print(utc_time)

		# Use time to determine relative orientation of J200
		etime = sp.str2et(utc_time)

		# Rotation
		q_J2000_SP = Quaternion()
		q_MOON_SP = Quaternion()
		q_J2000_SP.fromArray(state[4:8])
		R_J2000_SP = q_J2000_SP.toDCM()
		R_J200_MOON = sp.pxform("J2000","MOON_ME",etime)
		R_MOON_SP = R_J200_MOON.T@R_J2000_SP
		q_MOON_SP.fromDCM(R_MOON_SP)
		out_entry[4:8] = q_MOON_SP.toArray()
		#print(qJ2000toMCMF)

		# Position
		stateJ2000 = np.array([state[1],state[2],state[3]],dtype=float)
		P_ES_MOON_ME = R_J200_MOON@stateJ2000
		[P_ME_MOON_ME,_] = sp.spkpos("EARTH",etime,"MOON_ME","NONE","MOON")
		P_MS_MOON_ME = np.array((P_ME_MOON_ME+P_ES_MOON_ME))*1000
		out_entry[1:4] = P_MS_MOON_ME

		out_data.append(out_entry)

	return out_data

In [ ]:
# Setup data to be read
filefrom = join(BASE_DIR,"scratch","data","surface_poses.csv")
columns = {"x":5,"y":6,"z":7,"qs":16,"qx":17,"qy":18,"qz":19,"time":2}
fileto = join(BASE_DIR,"configs","trajectories","match0.csv")

data = read_data(filefrom,columns,header=False)
converted_states = convertEarthJ2000toMCMF(data)
state_text = ""
for state in converted_states:
	print(state)
	str_state = np.array(state,dtype=str)
	state_text += "{}\n".format(",".join(str_state))
with open(fileto,"w+") as f:
	f.write(state_text)